In [1]:
import keras
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16

C:\Users\admin\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
conv_base = VGG16(include_top=False, input_shape=(240,320,3))

In [23]:
base_dir = 'data/images'
train_dir = os.path.join(base_dir,'train')
val_dir = os.path.join(base_dir,'val')
test_dir = os.path.join(base_dir,'test')

In [4]:
datagen = ImageDataGenerator(rescale=1./255)

In [32]:
def extract_features(dir, sample_count, batch_size):
    # temporarili save feture_batches and label_batches
    features = []
    labels =[]
    generator = datagen.flow_from_directory(
        dir, 
        target_size=(240,320),
        batch_size=batch_size,
        class_mode='categorical')
    i = 0
    for inputs_batch, labels_batch in generator:
        feature_batch = conv_base.predict(inputs_batch)
        features.append(feature_batch)
        labels.append(labels_batch)        
        i += 1
        if i * batch_size >= sample_count:
            break
    features = np.concatenate(features)
    labels = np.concatenate(labels)
    return features, labels, generator.class_indices

In [33]:
n_train = 827
n_val = 273
n_test = 273

In [34]:
# train_features, train_labels = extract_features(train_dir, n_train)
train_features, train_labels, class_indices = extract_features(train_dir, n_train, 32)
val_features, val_labels, _ = extract_features(val_dir, n_val, 32)
test_features, test_labels, _ = extract_features(test_dir, n_test, 32)

Found 827 images belonging to 4 classes.
Found 273 images belonging to 4 classes.
Found 273 images belonging to 4 classes.


In [39]:
class_indices

{'empty': 0, 'paper': 1, 'rock': 2, 'scissor': 3}

In [40]:
np.savez('extracted_features&labels.npz',
         train_features=train_features,
         train_labels=train_labels,
         test_features=test_features,
         test_labels=test_labels,
         val_features=val_features,
         val_labels=val_labels,
         class_indices=class_indices)